In [35]:
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator
from datetime import datetime
from sklearn.metrics import *
from sklearn.metrics.pairwise import *
from collections import Counter
# import matplotlib相關套件
import matplotlib.pyplot as plt
# import字型管理套件
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot
from operator import itemgetter
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [2]:
#  import sys,os
print(sys.executable)
print(sys.version)
print(sys.version_info)
import findspark
# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
# conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.setMaster("local[12]")
conf.set("spark.driver.memory","265g") #滿載165G/
conf.set("spark.executor.memory", "110g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","105g")
conf.set("spark.driver.extraJavaOptions","-Xss95g")
conf.set("spark.driver.extraJavaOptions","-Xms95g")
# conf.set("spark.memory.storageFraction","0.2")
# conf.set("spark.driver.maxResultSize","0")
# conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.network.timeout","1500s")
conf.set("spark.sql.shuffle.partitions",140000)
conf.set("spark.driver.maxResultSize", "170g")

# spark = SparkSession.builder.appName("pcap_analyzer").config(conf=conf).getOrCreate() #pcap_analyzer pcapAnalyzer
spark = SparkSession.builder.master("local").appName("Leo_spark_123").config(conf=conf).getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.conf.set("spark.sql.debug.maxToStringFields", 300000)
spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",120*1024*1024*1024)

import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__,spark.version)
spark

/home/antslab/anaconda3/bin/python
3.7.10 (default, Feb 26 2021, 18:47:35) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)
1.7.0 3.0.0


In [3]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
# date_li.reverse()
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
# protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']

# isp_li.reverse()
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))
# port_df = pd.DataFrame() #1st time
port_df = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/all_port_nopayload_dist.pkl','rb'))
port_df

,country,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols
20200106_中嘉寬頻_0,"[Denmark, Netherlands]","[1, 1]","[1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]"
20200106_中嘉寬頻_1,"[France, France, Russia, Russia, Russia, Franc...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_中嘉寬頻_2,"[Russia, Russia, Russia, Denmark]","[2, 2, 2, 1]","[1, 1, 1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_中嘉寬頻_3,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_中嘉寬頻_4,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
...,...,...,...,...,...
20200112_中華電信_65531,"[Russia, Russia, Russia, Russia, Russia, Russi...","[1, 2, 2, 2, 2, 2, 2, 1, 2, 3, 1, 1, 2, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200112_中華電信_65532,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200112_中華電信_65533,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200112_中華電信_65534,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."


In [ ]:
for time in tqdm(date_li):
    for isp in isp_li:
        if time+'_'+isp in processed_li_rev:
            continue #之前有一部分已經跑過了        
        print("=== Now is about to run:",time,isp,"===")
        in_file = 'hdfs://192.168.50.123:8020/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
#         in_file = 'hdfs://192.168.50.200/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
        kdf = ks.read_parquet(in_file)
        kdf_nopayload = kdf[(kdf.tcp_i_payload_list.astype(str)=="[]")|(kdf.tcp_o_payload_list.astype(str)=="[]")]
        kdf_nopayload = kdf_nopayload[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','frame_i_max_protocols','frame_o_max_protocols']]
        gc.collect()
        kdf_nopayload = kdf_nopayload.to_pandas()
        kdf_gby_co = kdf_nopayload.groupby('tcp_dstport').agg(list)
        t_index = kdf_gby_co.index.tolist()
        t_index = [time+'_'+isp+'_'+str(int(ind)) for ind in t_index]
        kdf_gby_co.index = t_index        
        port_df = port_df.append(kdf_gby_co)
        gc.collect()
        pickle.dump(obj=port_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/all_port_nopayload_dist.pkl','wb'),protocol=pickle.HIGHEST_PROTOCOL)
port_df        

  0%|          | 0/7 [00:00<?, ?it/s]

=== Now is about to run: 20200112 凱擘 ===


In [33]:
processed_li = port_df.index.tolist()
processed_li_rev = ["_".join(x.split('_')[:2]) for x in processed_li]
processed_li_rev = list(set(processed_li_rev))
processed_li_rev

['20200106_台灣基礎開發',
 '20200108_凱擘',
 '20200109_台灣之星',
 '20200112_中嘉寬頻',
 '20200107_台灣大哥大',
 '20200110_台灣固網',
 '20200108_中華電信',
 '20200110_凱擘',
 '20200107_台灣碩網',
 '20200108_台灣之星',
 '20200109_中嘉寬頻',
 '20200110_台灣之星',
 '20200110_中華電信',
 '20200110_亞太電信',
 '20200111_台灣基礎開發',
 '20200106_台灣大哥大',
 '20200106_台固媒體',
 '20200106_凱擘',
 '20200109_台灣固網',
 '20200111_台固媒體',
 '20200110_台灣碩網',
 '20200106_亞太電信',
 '20200112_台灣大哥大',
 '20200106_台灣碩網',
 '20200111_中華電信',
 '20200106_中華電信',
 '20200107_中嘉寬頻',
 '20200111_亞太電信',
 '20200110_台灣基礎開發',
 '20200109_凱擘',
 '20200108_台灣固網',
 '20200107_台灣之星',
 '20200109_台灣大哥大',
 '20200108_亞太電信',
 '20200111_台灣大哥大',
 '20200112_中華電信',
 '20200108_台灣大哥大',
 '20200106_台灣之星',
 '20200109_台固媒體',
 '20200111_台灣之星',
 '20200111_中嘉寬頻',
 '20200110_中嘉寬頻',
 '20200107_亞太電信',
 '20200111_台灣固網',
 '20200111_台灣碩網',
 '20200109_亞太電信',
 '20200109_台灣碩網',
 '20200112_台灣固網',
 '20200107_中華電信',
 '20200112_台固媒體',
 '20200107_台固媒體',
 '20200108_中嘉寬頻',
 '20200110_台固媒體',
 '20200106_台灣固網',
 '20200107_台灣基礎開發',
 '20

In [10]:
c=0
for time in tqdm(date_li):
    for isp in isp_li:
        if time+'_'+isp in processed_li_rev:
            c+=1
            continue #之前有一部分已經跑過了     
c

100%|██████████| 7/7 [00:00<00:00, 42003.04it/s]


62

In [9]:
sorted(['a','b','c'],reverse=True)

['c', 'b', 'a']

In [16]:
%%time
def get_port(idx):
    return idx.split('_')[-1]
def get_proto(protocol_li):
    taken = []
    protocol_li_rev = []
    defined_protos = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip','tds',
                 'data','ssl','sock','rtsp','gsm_ipa','nbss','giop','ldap','mptcp','irc','ecatf','mqtt','ams','tfp',
                'amqp','x11','bittorrent','ndps','vnc','h501','dns']
    for x in processed_li:
        try:
            x = x.split(':')[3:]
        except AttributeError:
            continue
        x  = sorted(list(set(x)),reverse=True)
        protocol_li_rev.append(x)
#     protocol_li_rev = [sorted(list(set(x.split(':')[3:])),reverse=True) for x in protocol_li]
    protocol_li_rev = [" ".join(x) for x in protocol_li_rev]
    for pro_str in protocol_li_rev:
        for proto in defined_protos:
            if proto in pro_str:
                final = proto
                break
            else:
                final = pro_str
        taken.append(final)
    return list(set(taken))
def set2list(li):
    return list(set(li))
# port_df['port_num'] = port_df.index.map(get_port)
# port_df['in_pkts'] = port_df.session_i_tt_packet.apply(sum)
# port_df['out_pkts'] = port_df.session_o_tt_packet.apply(sum)
# port_df['in_proto'] = port_df.frame_i_max_protocols.map(get_proto)
# port_df['out_proto'] = port_df.frame_o_max_protocols.map(get_proto)
# port_df['country_set'] = port_df.country.apply(set2list)
port_df['session_num'] = port_df.session_i_tt_packet.apply(len)
port_df

CPU times: user 624 ms, sys: 74.3 ms, total: 699 ms
Wall time: 836 ms


,country,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols,port_num,in_pkts,out_pkts,country_set,session_num
20200106_中嘉寬頻_0,"[Denmark, Netherlands]","[1, 1]","[1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp]",0,2,2,"{Netherlands, Denmark}",2
20200106_中嘉寬頻_1,"[France, France, Russia, Russia, Russia, Franc...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",1,927,470,"{Canada, Thailand, United States, Germany, Sin...",470
20200106_中嘉寬頻_2,"[Russia, Russia, Russia, Denmark]","[2, 2, 2, 1]","[1, 1, 1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",2,7,4,"{Denmark, Russia}",4
20200106_中嘉寬頻_3,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",3,102,52,"{Denmark, Russia}",52
20200106_中嘉寬頻_4,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",4,475,238,"{Denmark, Russia}",238
...,...,...,...,...,...,...,...,...,...,...
20200112_中華電信_65531,"[Russia, Russia, Russia, Russia, Russia, Russi...","[1, 2, 2, 2, 2, 2, 2, 1, 2, 3, 1, 1, 2, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65531,1497,826,"{United States, Germany, France, China, Russia...",759
20200112_中華電信_65532,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65532,1785,957,"{United States, France, China, Russia}",918
20200112_中華電信_65533,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65533,1858,1019,"{Saudi Arabia, United States, France, China, R...",993
20200112_中華電信_65534,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",65534,1971,1034,"{United States, Sweden, China, Russia, France}",1004


In [17]:
%%time
port_df_rev = port_df[['port_num','session_num','in_pkts','out_pkts','country_set']]
port_df_rev['country_set'] = port_df_rev['country_set'].apply(list)
port_df_rev_gby = port_df_rev.groupby(['port_num']).agg(list)
port_df_rev_gby

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


CPU times: user 10.7 s, sys: 3.11 s, total: 13.8 s
Wall time: 15.8 s


,session_num,in_pkts,out_pkts,country_set
port_num,,,,
0,"[2, 13, 155, 16, 8, 1, 311, 1, 234, 60, 10, 14...","[2, 13, 310, 16, 15, 1, 613, 1, 455, 120, 10, ...","[2, 13, 155, 15, 8, 1, 302, 1, 239, 60, 10, 14...","[[Netherlands, Denmark], [Estonia, Netherlands..."
1,"[470, 2065, 515, 2118, 1664, 1369, 1024, 177, ...","[927, 4028, 1017, 4180, 3288, 2696, 2027, 348,...","[470, 2065, 515, 2112, 1666, 1369, 1024, 176, ...","[[Canada, Thailand, United States, Germany, Si..."
10,"[157, 486, 551, 1079, 330, 539, 24, 212, 651, ...","[313, 924, 1098, 2145, 659, 1068, 47, 414, 127...","[157, 486, 551, 1079, 330, 537, 24, 212, 651, ...","[[Russia], [Russia], [Russia], [Russia], [Russ..."
100,"[160, 1451, 235, 1856, 467, 951, 387, 31, 387,...","[305, 2428, 384, 3226, 682, 1636, 646, 58, 694...","[160, 1451, 235, 1856, 467, 951, 387, 31, 387,...","[[Canada, Japan, Thailand, United States, Neth..."
1000,"[700, 3240, 602, 2490, 2000, 1709, 1165, 185, ...","[1379, 5989, 1181, 4610, 3687, 3097, 2317, 354...","[699, 3240, 730, 2490, 1999, 1709, 1421, 184, ...","[[Japan, United States, Singapore, Netherlands..."
...,...,...,...,...
9995,"[16, 354, 66, 1004, 761, 221, 120, 36, 66, 478...","[28, 388, 68, 1681, 1333, 273, 142, 71, 76, 69...","[16, 354, 66, 1004, 761, 221, 120, 35, 66, 478...","[[Netherlands, Denmark], [France, Netherlands,..."
9996,"[14, 493, 139, 334, 366, 225, 121, 9, 66, 273,...","[25, 666, 247, 348, 536, 242, 144, 18, 68, 289...","[14, 493, 139, 334, 366, 225, 121, 9, 66, 273,...","[[Netherlands, Denmark], [France, Netherlands,..."
9997,"[790, 4095, 906, 4114, 4260, 3631, 1838, 193, ...","[1485, 7442, 1666, 7476, 7916, 6964, 3402, 382...","[790, 4094, 906, 4113, 4260, 3630, 1832, 193, ...","[[Thailand, United States, Germany, Singapore,..."


In [22]:
def count_country(lioli):
    return len(set(functools.reduce(operator.iconcat, lioli, [])))
port_df_rev_gby['session_num'] = port_df_rev_gby.session_num.apply(sum)
port_df_rev_gby['in_pkts'] = port_df_rev_gby['in_pkts'].apply(sum)
port_df_rev_gby['out_pkts'] = port_df_rev_gby['out_pkts'].apply(sum)
port_df_rev_gby['country_num'] = port_df_rev_gby['country_set'].apply(count_country)
port_df_rev_gby

,session_num,in_pkts,out_pkts,country_set,country_num
port_num,,,,,
0,4541,10606,6423,"[[Netherlands, Denmark], [Estonia, Netherlands...",23
1,54301,105773,54214,"[[Canada, Thailand, United States, Germany, Si...",21
10,41105,78770,41025,"[[Russia], [Russia], [Russia], [Russia], [Russ...",16
100,44454,79351,44960,"[[Canada, Japan, Thailand, United States, Neth...",21
1000,69003,124749,73227,"[[Japan, United States, Singapore, Netherlands...",21
...,...,...,...,...,...
9995,28353,54408,28259,"[[Netherlands, Denmark], [France, Netherlands,...",14
9996,19113,36124,19015,"[[Netherlands, Denmark], [France, Netherlands,...",12
9997,166348,311873,166563,"[[Thailand, United States, Germany, Singapore,...",21


In [26]:
port_df_rev_gby.index = port_df_rev_gby.index.astype(int)
port_df_rev_gby.sort_index(inplace=True)

In [30]:
port_df_rev_gby['session_portion'] = round((port_df_rev_gby['session_num'] / port_df_rev_gby['session_num'].sum())*100,4)
port_df_rev_gby

,session_num,in_pkts,out_pkts,country_set,country_num,session_portion
port_num,,,,,,
0,4541,10606,6423,"[[Netherlands, Denmark], [Estonia, Netherlands...",23,0.0003
1,54301,105773,54214,"[[Canada, Thailand, United States, Germany, Si...",21,0.0037
2,44920,80836,44976,"[[Denmark, Russia], [Russia], [Russia], [Russi...",15,0.0031
3,35564,69334,35469,"[[Denmark, Russia], [Russia], [Russia], [Russi...",18,0.0024
4,32762,64218,32843,"[[Denmark, Russia], [Russia], [Russia], [Germa...",16,0.0022
...,...,...,...,...,...,...
65531,30077,59366,30016,"[[Netherlands, Indonesia, Russia], [Russia], [...",18,0.0021
65532,15628,30549,15463,"[[Russia], [Russia], [Egypt], [Russia], [India...",15,0.0011
65533,57109,112877,93858,"[[India, China], [Greece, Indonesia, China, Al...",38,0.0039


In [32]:
port_df_rev_gby_save = port_df_rev_gby[['session_num','session_portion','in_pkts','out_pkts','country_num']]
port_df_rev_gby_save.to_excel('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pictures/nopayload_dist.xlsx')

In [31]:
port_df_rev_gby.to_excel('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pictures/nopayload_dist.xlsx')

KeyboardInterrupt: 

In [27]:
pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/all_port_nopayload_dist.pkl','rb'))

KeyboardInterrupt: 

In [4]:
time = date_li[0]
isp = isp_li[1]
# in_file = 'hdfs://192.168.50.123:8020/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
in_file = 'hdfs://192.168.50.200/user/hdfs/parquet/'+str(time[:4])+'_'+str(time[4:6])+'_'+str(time[6:])+'/'+isp+'/session_parquet/'+str(time)+'_session.parquet'
kdf = ks.read_parquet(in_file)
kdf

,session_time,session_time_list,session_duration,session_tt_packet,session_i_tt_packet,session_o_tt_packet,session_tt_frame_length,session_i_tt_frame_length,session_o_tt_frame_length,udp_tt_length,udp_i_tt_length,udp_o_tt_length,udp_i_avg_length,udp_o_avg_length,icmp_tt_length,icmp_i_tt_length,icmp_o_tt_length,icmp_i_avg_length,icmp_o_avg_length,icmp_tt_original_datagram_length,icmp_i_tt_datagram_length,icmp_o_tt_datagram_length,icmp_i_avg_datagram_length,icmp_o_avg_datagram_length,tcp_hdr_len_sum,tcp_tt_payload_length,tcp_i_tt_payload_length,tcp_o_tt_payload_length,tcp_i_avg_payload_length,tcp_o_avg_payload_length,ip_src,ip_dst,ip_proto,tcp_srcport,tcp_dstport,udp_srcport,udp_dstport,country,domain,isp,frame_i_common_protocols,frame_o_common_protocols,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
0,1.578262e+09,"[1578261951.454916, 1578261951.455137, 1578261...",0.150866,3,2,1,172,112,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,0,0,0.0,0.000000,0.0,1.0.166.181,210.202.233.172,6,33048,80,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,[],[]
1,1.578301e+09,"[1578301487.358119, 1578301487.358309, 1578301...",0.159539,10,5,5,1043,737,306.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,455,455,0.0,91.000000,0.0,1.0.182.188,210.63.253.114,6,63528,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb:smb:smb,eth:ethertype:ip:tcp,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[]
2,1.578302e+09,"[1578302056.393877, 1578302056.394066, 1578302...",0.575320,4,2,2,252,132,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,0,0,0.0,0.000000,0.0,1.0.182.188,210.63.253.164,6,56765,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,[],[]
3,1.578302e+09,"[1578302138.167121, 1578302138.167332, 1578302...",0.607414,4,2,2,252,132,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,0,0,0.0,0.000000,0.0,1.0.182.188,210.63.253.240,6,65273,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,[],[]
4,1.578302e+09,"[1578302150.016509, 1578302150.016778, 1578302...",0.594720,4,2,2,252,132,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,0,0,0.0,0.000000,0.0,1.0.182.188,210.63.253.251,6,50161,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,[],[]
5,1.578300e+09,"[1578300487.641315, 1578300487.641518, 1578300...",0.146228,7,4,3,414,228,186.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164,0,0,0.0,0.000000,0.0,1.0.182.188,210.63.253.70,6,58693,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,[],[]
6,1.578302e+09,"[1578302007.441837, 1578302007.442067, 1578302...",0.074844,6,3,3,448,262,186.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144,88,88,0.0,29.333333,0.0,1.0.187.191,210.200.17.101,6,1262,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp,[[3:11QRfDp03vV1V1Ud2FV1V1V1V1V1V1V1V1V1V1V1V1...,[]
7,1.578302e+09,"[1578302327.548338, 1578302327.548629, 1578302...",0.081596,6,3,3,497,311,186.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144,137,137,0.0,45.666667,0.0,1.0.187.191,210.200.17.123,6,9393,445,NaN,NaN,Thailand,totinternet.net,None,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[]
8,1.578301e+09,"[1578301032.90306, 1578301032.90335, 157830103...",7.062824,11,9,2,1234,1114,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236,616,616,0.0,68.444444,

In [5]:
kdf_nopayload = kdf[(kdf.tcp_i_payload_list.astype(str)=="[]")|(kdf.tcp_o_payload_list.astype(str)=="[]")]
kdf_nopayload = kdf_nopayload[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','frame_i_max_protocols','frame_o_max_protocols']]
kdf_nopayload

,country,tcp_dstport,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols
0,Thailand,80,2,1,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp
1,Thailand,445,5,5,eth:ethertype:ip:tcp:nbss:smb:smb:smb,eth:ethertype:ip:tcp
2,Thailand,445,2,2,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp
3,Thailand,445,2,2,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp
4,Thailand,445,2,2,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp
5,Thailand,445,4,3,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp
6,Thailand,445,3,3,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp
7,Thailand,445,3,3,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp
8,Thailand,445,9,2,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp
9,Thailand,445,3,0,eth:ethertype:ip:tcp,None


In [12]:
kdf_nopayload = kdf_nopayload.to_pandas()
kdf_gby_co = kdf_nopayload.groupby('tcp_dstport')
# key_di = {}
# keys = kdf_nopayload.columns.tolist()
# for k in keys:
#     if k=='tcp_dstport':
#         continue
#     key_di[k] = 'collect_list'
kdf_gby_co = kdf_gby_co.agg(list)
kdf_gby_co

,country,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols
tcp_dstport,,,,,
0.0,"[Estonia, Netherlands, Estonia, Estonia, Nethe...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
1.0,"[Russia, Russia, Russia, Russia, France, Franc...","[2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
2.0,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
3.0,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
4.0,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
...,...,...,...,...,...
65527.0,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
65529.0,"[China, China, China, China, China, China, Chi...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
65530.0,[Ukraine],[1],[0],[eth:ethertype:ip:icmp:ip:tcp],[None]


In [13]:
# kdf_gby_co = kdf_gby_co.to_pandas()
da=time
t_index = kdf_gby_co.index.tolist()
t_index = [da+'_'+isp+'_'+str(int(ind)) for ind in t_index]
kdf_gby_co.index = t_index
kdf_gby_co
# temp = kdf_gby_co.reset_index(drop=True)
# temp

,country,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols
20200106_亞太電信_0,"[Estonia, Netherlands, Estonia, Estonia, Nethe...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_1,"[Russia, Russia, Russia, Russia, France, Franc...","[2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_2,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_3,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_4,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
...,...,...,...,...,...
20200106_亞太電信_65527,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_65529,"[China, China, China, China, China, China, Chi...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_65530,[Ukraine],[1],[0],[eth:ethertype:ip:icmp:ip:tcp],[None]
20200106_亞太電信_65533,"[China, China, China, China, China, China, Chi...","[4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."


In [14]:
port_df = pd.DataFrame()
# port_kdf.index.rename("tcp_dstport")
port_df = port_df.append(kdf_gby_co)
port_df

,country,session_i_tt_packet,session_o_tt_packet,frame_i_max_protocols,frame_o_max_protocols
20200106_亞太電信_0,"[Estonia, Netherlands, Estonia, Estonia, Nethe...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_1,"[Russia, Russia, Russia, Russia, France, Franc...","[2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_2,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_3,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_4,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
...,...,...,...,...,...
20200106_亞太電信_65527,"[Russia, Russia, Russia, Russia, Russia, Russi...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_65529,"[China, China, China, China, China, China, Chi...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."
20200106_亞太電信_65530,[Ukraine],[1],[0],[eth:ethertype:ip:icmp:ip:tcp],[None]
20200106_亞太電信_65533,"[China, China, China, China, China, China, Chi...","[4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e..."


In [ ]:

# take_columns = ['session_i_tt_packet','session_o_tt_packet']
pkt_sess_df = pd.DataFrame(columns=isp_li,index=date_li)
country_count_dict = {}
protocol_count_dict = {}
tri_country_df = pd.DataFrame()
port_dist_df = pd.DataFrame()
# counter = 0
for da in tqdm(date_li):
    for isp in isp_li:
        print("=== Now is about to run:",da,isp,"===")
        gc.collect()
        dir_path = "/mnt/Raid160TB/pcap_inter/"+str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])+"/"+isp+"/intermeidate_data/"
        saved_path = dir_path+"df2_tuples_"+isp+".pkl"
        df2,pickle_dir,picture_dir = pickle.load(open(saved_path,'rb')) 
        
        temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','tcp_i_tt_payload_length']]
        temp1 = temp1_master.groupby(['country'])
        temp1 = temp1.aggregate(list)
        temp11 = temp1[temp1.index.isin(['China','Taiwan','Ireland'])] #指定需要的國家
        t_index = temp11.index.tolist()
        t_index = [da+'_'+isp+'_'+ind for ind in t_index]
        temp11.index = t_index
        tri_country_df = tri_country_df.append(temp11)
        
        temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','tcp_i_tt_payload_length','tcp_o_tt_payload_length','frame_i_max_protocols','frame_o_max_protocols','tcp_i_payload_list','tcp_o_payload_list']]
        temp2 = temp1_master.groupby(['tcp_dstport'])
        temp2 = temp2.aggregate(list)
        t_index = temp2.index.tolist()
        t_index = [da+'_'+isp+'_'+str(ind) for ind in t_index]
        temp2.index = t_index
        port_dist_df = port_dist_df.append(temp2)
        gc.collect()
        #checkpoint
        pickle.dump(obj=tri_country_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/IrelandCnTw_beforepreprocess_dist.pkl','wb'))
        pickle.dump(obj=port_dist_df,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/all_port_beforepreprocess_dist.pkl','wb'))
#         break
#     break
# print(df2.columns) #tcp_dstport、session_duration、session_i_tt_packet、tcp_i_tt_payload_length、
# df2
port_dist_df

  0%|          | 0/7 [00:00<?, ?it/s]

=== Now is about to run: 20200106 中嘉寬頻 ===
=== Now is about to run: 20200106 亞太電信 ===
=== Now is about to run: 20200106 台固媒體 ===
=== Now is about to run: 20200106 台灣固網 ===
=== Now is about to run: 20200106 台灣大哥大 ===
=== Now is about to run: 20200106 中華電信 ===


In [19]:
temp1 = df2[['country','tcp_dstport','session_i_tt_packet','tcp_i_tt_payload_length']]
temp1 = temp1.groupby(['country'])
temp1 = temp1.aggregate(list)
temp1
# df2[df2.country == 'China'] #Ireland、China、Taiwan

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
country,,,
Afghanistan,[80],[177],[185]
Albania,"[8080, 80, 80, 80, 8080, 80, 8080, 80, 80, 80,...","[6, 7, 21, 11, 6, 69, 6, 7, 6, 22, 24, 242, 36...","[185, 186, 13909, 192, 192, 14533, 187, 185, 1..."
Algeria,"[445, 445, 445, 445, 445, 445, 445, 445, 445, ...","[6, 6, 6, 6, 8, 8, 13, 54, 8, 6, 6, 8, 6, 8, 8...","[364, 364, 364, 364, 455, 455, 866, 63938, 455..."
Angola,[22],[18],[1175]
Antigua and Barbuda,"[22, 22, 22]","[17, 17, 18]","[1175, 1175, 1079]"
...,...,...,...
Vietnam,"[445, 2222, 2222, 445, 2222, 2222, 22, 22, 22,...","[12, 13, 13, 6, 13, 13, 13, 14, 8, 14, 40, 10,...","[592, 713, 713, 364, 713, 713, 729, 932, 651, ..."
Yemen,"[445, 445, 445, 445, 445, 445, 445, 445, 445, ...","[4, 8, 8, 6, 8, 8, 8, 8, 8, 8, 13, 8, 18, 8, 6...","[191, 364, 455, 364, 455, 455, 442, 364, 455, ..."
Zambia,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[13, 13, 13, 16, 12, 13, 12, 13, 12, 15, 13, 1...","[713, 713, 713, 713, 745, 713, 745, 713, 745, ..."


In [28]:
temp11 = temp1[temp1.index.isin(['China','Taiwan','Ireland'])]
t_index = temp11.index.tolist()
t_index = [da+'_'+isp+'_'+ind for ind in t_index]
temp11.index = t_index
temp11

,tcp_dstport,session_i_tt_packet,tcp_i_tt_payload_length
20200106_中嘉寬頻China,"[1433, 1433, 1433, 1433, 1433, 1433, 1433, 143...","[6, 8, 6, 7, 6, 7, 8, 8, 7, 8, 8, 7, 6, 6, 6, ...","[300, 398, 347, 400, 351, 349, 351, 398, 408, ..."
20200106_中嘉寬頻Ireland,"[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[16, 15, 14, 14, 14, 14, 14, 31, 14, 14, 14, 1...","[713, 713, 1312, 1312, 1312, 1312, 1312, 3264,..."
20200106_中嘉寬頻Taiwan,"[445, 21, 21, 21, 21, 22, 22, 2222, 2222, 2222...","[56, 7, 7, 7, 7, 14, 14, 14, 16, 17, 16, 15, 1...","[63938, 33, 25, 31, 29, 886, 886, 886, 886, 88..."


In [46]:
temp1_master = df2[['country','tcp_dstport','session_i_tt_packet','session_o_tt_packet','tcp_i_tt_payload_length','tcp_o_tt_payload_length','frame_i_max_protocols','frame_o_max_protocols','tcp_i_payload_list','tcp_o_payload_list']]
temp2 = temp1_master.groupby(['tcp_dstport'])
temp2 = temp2.aggregate(list)
t_index = temp2.index.tolist()
t_index = [da+'_'+isp+'_'+str(ind) for ind in t_index]
temp2.index = t_index
temp2

,country,session_i_tt_packet,session_o_tt_packet,tcp_i_tt_payload_length,tcp_o_tt_payload_length,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
20200106_中嘉寬頻_21,"[United States, Taiwan, Taiwan, Taiwan, Taiwan...","[6, 7, 7, 7, 7, 7, 7, 7, 8, 11, 11, 11, 5, 8, ...","[6, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10...","[10, 33, 25, 31, 29, 24, 33, 28, 41, 34, 37, 3...","[51, 113, 113, 113, 113, 113, 113, 113, 182, 1...","[eth:ethertype:ip:tcp:ftp, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:ftp, eth:ethertype:ip:tc...","[[[3:2f0/9B9En:2f0/j9En, 1578249914.441039000,...",[[[3:QCWVSsTQCfyNTQ/OP/1Tpv9duESXjm:QCSSsTbJmd...
20200106_中嘉寬頻_22,"[Thailand, South Korea, United States, China, ...","[13, 13, 13, 13, 13, 14, 14, 10, 13, 14, 14, 1...","[15, 15, 15, 15, 15, 15, 15, 16, 15, 15, 19, 1...","[729, 713, 713, 713, 729, 886, 729, 509, 713, ...","[2092, 2092, 2092, 2092, 2092, 2044, 2092, 486...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:ssh, eth:ethertype:ip:tc...","[[[3:DfQMPmfK7PATaupm:DPQKtupm, 1578254105.213...",[[[3:DfQMPmflfTAfQW4Y7wUZIr6drA60vA6Eia1rm:DPQ...
20200106_中嘉寬頻_23,"[Romania, Romania, Romania, Romania, Romania, ...","[6, 6, 6, 6, 13, 6, 6, 6, 13, 6, 10, 6, 6, 5, ...","[5, 5, 5, 5, 7, 5, 5, 5, 6, 4, 6, 5, 4, 4, 5, ...","[33, 33, 4, 4, 24, 4, 4, 4, 954, 8, 16, 33, 48...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[eth:ethertype:ip:tcp:telnet, eth:ethertype:ip...","[eth:ethertype:ip:tcp:telnet, eth:ethertype:ip...",[[[3:3D/UdfrzC5SW/J2DrV7/UCVfFfrfVJDd/:gdCf+ra...,"[[[3:UNT2CIQ7A4fTxx18uZhEn:UgNIrfdxSiEn, 15782..."
20200106_中嘉寬頻_25,"[Vietnam, China, Sweden, Poland, Poland, Polan...","[40, 7, 6, 6, 10, 12, 8, 5, 6, 6, 7, 8, 7, 7, ...","[22, 8, 5, 8, 14, 16, 12, 6, 8, 8, 10, 11, 10,...","[32178, 15, 27, 31, 35, 35, 35, 11, 32, 32, 29...","[135, 54, 40, 54, 64, 64, 64, 38, 54, 54, 56, ...",[eth:ethertype:ip:tcp:smtp:imf:data-text-lines...,"[eth:ethertype:ip:tcp:smtp, eth:ethertype:ip:t...",[[[3:RlhfRcfR6P+5EfRGYdfqIW+aBfu:RlJRe6Pd7qFpB...,"[[[3:QCWVSsTQCfyNTQ97E:QCSSsTbJpE, 1578325360...."
20200106_中嘉寬頻_42,"[Russia, Russia, Russia, Russia, Russia, Germa...","[8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 5, 5, 8, 8, 8, ...","[8, 7, 8, 8, 8, 7, 4, 7, 7, 7, 4, 5, 8, 8, 8, ...","[465, 465, 465, 465, 465, 465, 47, 465, 465, 4...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...","[eth:ethertype:ip:tcp, eth:ethertype:ip:tcp, e...",[[[3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/...,"[[[3:Sn:Sn, 1578284678.743698000, 2]], [[3:Sn:..."
20200106_中嘉寬頻_80,"[Thailand, China, China, India, Hong Kong, Ind...","[27, 38, 3473, 7, 6, 35, 5, 36, 122, 7, 6, 10,...","[16, 42, 2521, 6, 5, 29, 5, 22, 80, 6, 5, 5, 4...","[14986, 18596, 15544, 185, 256, 14419, 232, 14...","[14280, 48580, 17593647, 96, 446, 85432, 446, ...","[eth:ethertype:ip:tcp:http:urlencoded-form, et...","[eth:ethertype:ip:tcp:http:data-text-lines, et...",[[[24:uAawYD5K/S18YH+w18YHxwdN04adN/QDdlUX18YH...,[[[96:DVb4NF1Cp4QuQxr9sHjqozJnsnSnrnb01gH1FUyY...
20200106_中嘉寬頻_110,"[United States, United States, Romania, Nether...","[7, 7, 7, 7, 8, 7, 9, 7, 7, 7, 7, 5, 8, 12, 7,...","[6, 6, 6, 6, 6, 6, 10, 6, 6, 6, 6, 5, 7, 11, 6...","[10, 6, 18, 204, 18, 18, 71, 18, 18, 18, 18, 6...","[18, 18, 18, 18, 18, 18, 164, 18, 18, 18, 18, ...","[eth:ethertype:ip:tcp:pop, eth:ethertype:ip:tc...","[eth:ethertype:ip:tcp:pop:imf, eth:ethertype:i...","[[[3:2f0/9B9En:2f0/j9En, 1578240246.977626000,...","[[[3:QCWVRBf5jWfPSXjm:QCScfqa, 1578240246.8144..."
20200106_中嘉寬頻_111,"[Vietnam, Vietnam, Vietnam, Vietnam, Vietnam, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[120, 120, 120, 120, 120, 120, 120, 120, 120, ...","[64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 6...","[eth:ethertype:ip:tcp:rpc:portmap, eth:etherty...","[eth:ethertype:ip:tcp:rpc:portmap, eth:etherty...",[[[3:PVVfXWdu469V1V1V1V1V1VP/B1XV1VP/vV1VYvV1V...,[[[3:PVVf

In [26]:
Counter(temp1.loc['China']['tcp_dstport'])

Counter({1433: 69164,
         2222: 39694,
         80: 19158,
         22: 158906,
         445: 4017,
         3389: 1733,
         8080: 297,
         25: 131,
         21: 480,
         139: 2388,
         3306: 899,
         1023: 113,
         9999: 18,
         135: 70,
         443: 60,
         1025: 112,
         2954: 61,
         587: 14,
         3128: 16,
         1080: 5,
         143: 3,
         23: 2})

In [ ]:
dir_path = "/mnt/Raid160TB/pcap_inter/"+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+"/"+isp+"/intermeidate_data/"
df2,pickle_dir,picture_dir = pickle.load(open(dir_path+"df2_tuples_"+isp+".pkl",'rb'))
df2

In [ ]:
date_li = ['20200106','20200107','20200108','20200109','20200110',
           '20200111','20200112'] #改
date_li.reverse()
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
isp_li.reverse()
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
# take_columns = ['session_i_tt_packet','session_o_tt_packet']
pkt_sess_df = pd.DataFrame(columns=isp_li,index=date_li)
# country_count_dict = {}
protocol_inpktsess_dict = {}
for da in tqdm(date_li):
    for isp in isp_li:
        print("=== Now is about to run:",da,isp,"===")
        gc.collect()
        dir_path = "/mnt/Raid160TB/pcap_inter/"+str(da[:4])+'_'+str(da[4:6])+'_'+str(da[6:])+"/"+isp+"/intermeidate_data/"
        saved_path = dir_path+"df2_tuples_"+isp+".pkl"
        df2,pickle_dir,picture_dir = pickle.load(open(saved_path,'rb'))  
        for protocol_name in protocols_need:
            df2_protocol = df2[(df2.frame_i_max_protocols.str.contains(protocol_name))&(df2.frame_o_max_protocols.str.contains(protocol_name))]        
            df2_protocol['inbound_payload_pkts'] = df2_protocol['tcp_i_payload_list'].apply(len)
            session_num = len(df2_protocol)
            in_pkt_payload_num = df2_protocol['inbound_payload_pkts'].sum()
            protocol_inpktsess_dict[da+'_'+isp+'_'+protocol_name] = [in_pkt_payload_num,session_num]

        pickle.dump(obj=protocol_inpktsess_dict,file=open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/case_pickles/inbondpkt_sess_dist_dict_rev.pkl','wb'))

protocol_inpktsess_dict #inbond pkt.# , sess.#

  0%|          | 0/7 [00:00<?, ?it/s]

=== Now is about to run: 20200112 台灣碩網 ===


/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


=== Now is about to run: 20200112 台灣基礎開發 ===
=== Now is about to run: 20200112 台灣之星 ===
=== Now is about to run: 20200112 凱擘 ===
=== Now is about to run: 20200112 中華電信 ===
=== Now is about to run: 20200112 台灣大哥大 ===
=== Now is about to run: 20200112 台灣固網 ===
=== Now is about to run: 20200112 台固媒體 ===
=== Now is about to run: 20200112 亞太電信 ===
=== Now is about to run: 20200112 中嘉寬頻 ===


 14%|█▍        | 1/7 [22:49:37<136:57:44, 82177.44s/it]

=== Now is about to run: 20200111 台灣碩網 ===
=== Now is about to run: 20200111 台灣基礎開發 ===
=== Now is about to run: 20200111 台灣之星 ===
=== Now is about to run: 20200111 凱擘 ===
=== Now is about to run: 20200111 中華電信 ===
=== Now is about to run: 20200111 台灣大哥大 ===
=== Now is about to run: 20200111 台灣固網 ===
=== Now is about to run: 20200111 台固媒體 ===
=== Now is about to run: 20200111 亞太電信 ===
=== Now is about to run: 20200111 中嘉寬頻 ===


 29%|██▊       | 2/7 [45:23:00<113:20:22, 81604.48s/it]

=== Now is about to run: 20200110 台灣碩網 ===
=== Now is about to run: 20200110 台灣基礎開發 ===
=== Now is about to run: 20200110 台灣之星 ===
